In [35]:
!git clone https://github.com/google-research/bert.git

Cloning into 'bert'...


In [4]:
# # Download BERT BASE model from tF hub
# !wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
# !unzip the file
# !unzip uncased_L-12_H-768_A-12.zip

'wget' is not recognized as an internal or external command,
operable program or batch file.
'unzip' is not recognized as an internal or external command,
operable program or batch file.
'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [10]:
# %% tensorflow_version 1.x

In [36]:
import os 
import re 
import numpy as np 
import pandas as pd 
from sklearn.preprocessing import LabelEncoder 
from sklearn.model_selection import train_test_split 
import tensorflow as tf 
from tensorflow import keras 
import csv 
from sklearn import metrics 

In [37]:
df =pd.read_csv('Labelled_news_data.csv')
print(df.shape)
df.info()


(429, 6)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 429 entries, 0 to 428
Data columns (total 6 columns):
id       429 non-null int64
Firm     429 non-null object
Date     429 non-null object
Title    429 non-null object
label    429 non-null int64
Body     429 non-null object
dtypes: int64(2), object(4)
memory usage: 20.2+ KB


In [38]:
print(df.head(5))


   id  Firm       Date                                              Title  \
0   1  HDFC  05-Nov-20  Buy Housing Development Finance Corporation: t...   
1   2  HDFC  03-Nov-20  What should investors do with HDFC after Q2 re...   
2   3  HDFC  03-Nov-20  HDFC Standalone September 2020 Net Sales at Rs...   
3   4  HDFC  03-Nov-20  HDFC Consolidated September 2020 Net Sales at ...   
4   5  HDFC  02-Nov-20  HDFC Q2 standalone profit falls 28% YoY, but b...   

   label                                               Body  
0      1  Sharekhan's research repor on Housing Developm...  
1      1  Housing Development Finance Corporation (HDFC)...  
2      1  Reported Standalone quarterly numbers for Hous...  
3      1  Reported Consolidated quarterly numbers for Ho...  
4      1  HDFC on November 2 posted a 27.55 percent year...  


In [39]:
# shuffle the DataFrame rows 
df = df.sample(frac = 1) 
print(df.shape)
df.head(5)

(429, 6)


,id,Firm,Date,Title,label,Body
55,56,Tata Steel,13-Aug-20,Tata Steel likely to post a loss in Q1 as lock...,0,Tata Steel is expected to post a loss in the r...
310,311,Titan Company,30-Oct-20,Titan Company Consolidated September 2020 Net ...,0,Reported Consolidated quarterly numbers for Ti...
338,339,Sun Pharma,04-Nov-20,Sun Pharma Consolidated September 2020 Net Sal...,1,Reported Consolidated quarterly numbers for Su...
164,165,ITC,06-Nov-20,"ITC Q2 profit falls 20% to Rs 3,232 crore; FMC...",0,Cigarette-hotel-to-FMCG major ITC on November ...
56,57,ONGC,14-Dec-20,ONGC shares jump 23% in December so far; broke...,1,Shares of Oil and Natural Gas Corporation (ONG...


In [40]:
train =df[:int((len(df)+1)*.60)]
test =df[int((len(df)+1)*.60):]
print(train.shape)
print(test.shape)

(258, 6)
(171, 6)


In [41]:
train =pd.DataFrame(train)
test =pd.DataFrame(test)
train.head(5)

,id,Firm,Date,Title,label,Body
55,56,Tata Steel,13-Aug-20,Tata Steel likely to post a loss in Q1 as lock...,0,Tata Steel is expected to post a loss in the r...
310,311,Titan Company,30-Oct-20,Titan Company Consolidated September 2020 Net ...,0,Reported Consolidated quarterly numbers for Ti...
338,339,Sun Pharma,04-Nov-20,Sun Pharma Consolidated September 2020 Net Sal...,1,Reported Consolidated quarterly numbers for Su...
164,165,ITC,06-Nov-20,"ITC Q2 profit falls 20% to Rs 3,232 crore; FMC...",0,Cigarette-hotel-to-FMCG major ITC on November ...
56,57,ONGC,14-Dec-20,ONGC shares jump 23% in December so far; broke...,1,Shares of Oil and Natural Gas Corporation (ONG...


In [42]:
df_bert =pd.DataFrame({
    'user_id':train['id'],
            'label': train['label'],
            'alpha':['a']*train.shape[0],
            'text':train["Body"].replace(r'\n',' ',regex=True)
})
            
df_bert_train, df_bert_dev = train_test_split(df_bert, test_size=0.01)

# Creating test dataframe according to BERT
df_bert_test = pd.DataFrame({'User_ID':test['id'],
                 'text':test['Body'].replace(r'\n',' ',regex=True)})

In [43]:
df_bert_train.head(6)

,user_id,label,alpha,text
172,173,1,a,Prabhudas Lilladher's research report on ITCIT...
345,346,1,a,Dolat Capital's research report on Sun Pharmac...
373,374,1,a,Sharekhan's research repor on UltraTech Cement...
230,231,1,a,The country's largest carmaker Maruti Suzuki I...
346,347,1,a,KRChoksey's research report on Sun PharmaSun P...
402,403,1,a,ICICI Direct's research report on Tech Mahindr...


In [47]:
df_bert_train.to_csv('data/train.tsv', sep='\t', index=False, header=False)
df_bert_dev.to_csv('data/dev.tsv', sep='\t', index=False, header=False)
df_bert_test.to_csv('data/test.tsv', sep='\t', index=False, header=True)


In [45]:
# !python bert/run_classifier.py \
# --task_name=cola \
# --do_train=true \
# --do_eval=true \
# --do_predict=true \
# --data_dir=data \
# --vocab_file=uncased_L-12_H-768_A-12/vocab.txt \
# --bert_config_file=uncased_L-12_H-768_A-12/bert_config.json \
# --init_checkpoint=uncased_L-12_H-768_A-12/bert_model.ckpt \
# --max_seq_length=32 \
# --train_batch_size=1 \
# --learning_rate=2e-5 \
# --num_train_epochs=5 \
# --output_dir=bert_output \
# --do_lower_case = True \
# --save_checkpoints_steps=10000

Traceback (most recent call last):
  File "bert/run_classifier.py", line 25, in <module>
    import optimization
  File "C:\Users\King\Desktop\Untitled Folder\bert\optimization.py", line 87, in <module>
    class AdamWeightDecayOptimizer(tf.train.Optimizer):
AttributeError: module 'tensorflow_core._api.v2.train' has no attribute 'Optimizer'


In [48]:
!python bert/run_classifier.py \
--task_name=cola \
--do_predict=true \
--data_dir=data \
--vocab_file=uncased_L-12_H-768_A-12/vocab.txt \
--bert_config_file=uncased_L-12_H-768_A-12/bert_config.json \
--init_checkpoint=bert_output/model.ckpt-0 \
--max_seq_length=32 \
--output_dir=bert_output \

Traceback (most recent call last):
  File "bert/run_classifier.py", line 29, in <module>
    flags = tf.flags
AttributeError: module 'tensorflow' has no attribute 'flags'


In [15]:
# code 
import  csv 
label_results =[] 
with open('bert_output/test_results.tsv') as file: 
    rows = csv.reader(file, delimiter ="\t") 
    for row in rows: 
      data_1 =[float(i) for i in row] 
      label_results.append(data_1.index(max(data_1))) 

In [16]:
print("Accuracy", metrics.accuracy_score(test['label'], label_results)) 
print("F1-Score", metrics.f1_score(test['label'], label_results)) 

Accuracy 0.8421052631578947
F1-Score 0.9045936395759717


In [17]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test['label'],label_results)

array([[ 16,  19],
       [  8, 128]])